# Optimisation des hyperparamètres

## Utilisation de keras tuner

In [28]:
import keras_tuner as kt
import keras

In [29]:
(X_train,y_train),(X_test,y_test)=keras.datasets.fashion_mnist.load_data()

Création de la fonction appelée par le tuner d'hyperparamètres

In [30]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden",min_value=0,max_value=8,default=0)
    learning_rate = hp.Float("learning_rate",min_value=1e-4,max_value=1e-2,sampling="log") # par défaut, default = min_value # sampling log pour assurer une répartission égale entre 1e-4 et 1e-2
    n_neurons = hp.Int("n_neurons",min_value = 16,max_value = 256)
    optimizers = hp.Choice("optimizer",values = ["adam","sgd"])
    if optimizers == "sgd":
        optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model = keras.Sequential()
    model.add(keras.layers.Flatten())

    for _ in range(n_hidden):
        model.add(keras.layers.Dense(n_hidden,activation="relu"))
    
    model.add(keras.layers.Dense(10,activation="softmax"))

    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

Création d'un tuner qui cherche des combinaisons random d'hyperparamètres

In [31]:
random_tuner = kt.RandomSearch(build_model, #va chercher parmi les models spécifié par cette fn
                               objective="val_accuracy", # va classer en fonction de l'accuracy du jeu de validation
                               max_trials=5, # ne va pas faire plus de 5 tirages parmi tous les choix de models possible
                               overwrite=True, # Réinitialise les données à chaque fois qu'on appelle seach()
                               directory="my_fashion_mnist", # données rangées dans le dossier my_fashion_mnist
                               project_name="my_rnd_research", # dans un sous-répertoire my_rns_research
                               seed=42)

# ATTENTION!!!
# Il vaut mieux en général mettre overwrite = False comme ça, on garde les données précédente et cela permet de comparer les précédents modèles
# aux nouveaux quand on va reappeler la fonction search()

In [ ]:
random_tuner.search(X_train,y_train,epochs = 10, validation_split = 0.2)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
5                 |5                 |n_hidden
0.00011943        |0.00011943        |learning_rate
114               |114               |n_neurons
adam              |adam              |optimizer

Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 2.1843 - accuracy: 0.1922 - val_loss: 2.0240 - val_accuracy: 0.2559
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/step - loss: 1.9687 - accuracy: 0.2626 - val_loss: 1.9055 - val_accuracy: 0.2777
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 1.8725 - accuracy: 0.2790 - val_loss: 1.8184 - val_accuracy: 0.2882
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 1.7912 - accuracy: 0.2930 - val_loss: 1.7653 - val_accuracy: 0.2942
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 1.7322 - accuracy: 0.3064 - val_loss: 1.7099 - val_accuracy: 0.3090
Epoc

In [ ]:
# Avoir le meilleur model parmi les 3 premiers
top3_models = random_tuner.get_best_models(num_models=3)
best_model = top3_models[0]


In [ ]:
# Avoir les meilleurs params parmi les parmas des 3 models arrivés premiers
top3_params = random_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values #top3_params[0] est un objet et on appelle ses valeurs

{'n_hidden': 5,
 'learning_rate': 0.30841471469904036,
 'n_neurons': 114,
 'optimizer': 'adam'}

In [26]:
#Retourner les spécificités du meilleur model:
best_trial = random_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 0 summary
Hyperparameters:
n_hidden: 5
learning_rate: 0.30841471469904036
n_neurons: 114
optimizer: adam
Score: 0.10300000011920929
